In [1]:
import warnings
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

#supressing warnings for readability
warnings.filterwarnings("ignore")

# To plot pretty figures directly within Jupyter
%matplotlib inline

# pandas options
pd.set_option("display.max.columns", None)
pd.set_option("display.max.rows", None)
pd.set_option("display.precision", 2)

In [2]:
import pickle

with open('../data/var_store/X_resampled.pkl', 'rb') as f:
    X_resampled = pickle.load(f)

with open('../data/var_store/y_resampled.pkl', 'rb') as f:
    y_resampled = pickle.load(f)

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Create a decision tree classifier
tree = DecisionTreeClassifier()

# Train the decision tree classifier with class weighting
tree.fit(X_train, y_train)

# Make predictions on the test data
predictions = tree.predict(X_test)

# Evaluate the model performance
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

       False       0.78      0.76      0.77     70135
        True       0.77      0.79      0.78     70403

    accuracy                           0.77    140538
   macro avg       0.77      0.77      0.77    140538
weighted avg       0.77      0.77      0.77    140538



In [5]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, predictions))

0.7743599595838848


In [18]:
import optuna
import gc
import lightgbm as lgb
from optuna import Trial
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

def fitLGBM(trial,X, y):
    
    params={
      'n_estimators':trial.suggest_int('n_estimators', 0, 1000), 
      'num_leaves':trial.suggest_int('num_leaves', 2, 512),
      'max_depth':trial.suggest_int('max_depth', 2, 128),
      'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.15),
      'min_split_gain': trial.suggest_loguniform('min_split_gain', 0.001, 0.1),
      'feature_fraction':trial.suggest_uniform('feature_fraction',0.1, 1.0),
      'bagging_freq':trial.suggest_int('bagging_freq',0.1,10),
      'verbosity': -1,
      'random_state': 42
            }
    kfolds = KFold(n_splits=5, shuffle=True, random_state=42)
    model = LGBMClassifier(**params)
    
    res=[]
    for i, (tdx, vdx) in enumerate(kfolds.split(X, y)):
        X_train, X_valid, y_train, y_valid = X[tdx], X[vdx], y[tdx], y[vdx]
        model.fit(X_train, y_train,
                 eval_set=[(X_train, y_train), (X_valid, y_valid)],
                 early_stopping_rounds=30, verbose=False)
        preds = model.predict_proba(X_valid)
        res.append(roc_auc_score(y_valid, preds[:,1]))
    
    err = np.mean(res)
    
    return model, err

In [19]:
def objective(trial:Trial):
    
    gc.collect()
    models=[]
    validScore=0
   
    model,log = fitLGBM(trial,X_resampled,y_resampled)
    
    models.append(model)
    gc.collect()
    validScore+=log
    validScore/=len(models)
    
    return validScore

In [20]:
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
study.optimize(objective, timeout=60*60*2)

[I 2023-07-03 13:00:42,167] A new study created in memory with name: no-name-ccc78b99-3769-4835-977b-6e5f43d59d42


[LightGBM] [Warning] feature_fraction is set=0.42466694883715683, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.42466694883715683
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] feature_fraction is set=0.42466694883715683, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.42466694883715683
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] feature_fraction is set=0.42466694883715683, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.42466694883715683
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] feature_fraction is set=0.42466694883715683, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.42466694883715683
[LightGBM] [Warning] bagging_freq is set=10, s

[I 2023-07-03 13:05:05,274] Trial 0 finished with value: 0.870200334753946 and parameters: {'n_estimators': 567, 'num_leaves': 393, 'max_depth': 34, 'learning_rate': 0.024496052614050117, 'min_split_gain': 0.016672273427405817, 'feature_fraction': 0.42466694883715683, 'bagging_freq': 10}. Best is trial 0 with value: 0.870200334753946.


[LightGBM] [Warning] feature_fraction is set=0.22963230135342616, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.22963230135342616
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.22963230135342616, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.22963230135342616
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.22963230135342616, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.22963230135342616
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.22963230135342616, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.22963230135342616
[LightGBM] [Warning] bagging_freq is set=7, subsampl

[I 2023-07-03 13:10:16,461] Trial 1 finished with value: 0.8882495131389302 and parameters: {'n_estimators': 799, 'num_leaves': 316, 'max_depth': 45, 'learning_rate': 0.09431663160560344, 'min_split_gain': 0.0027046346428031185, 'feature_fraction': 0.22963230135342616, 'bagging_freq': 7}. Best is trial 1 with value: 0.8882495131389302.


...

[I 2023-07-03 14:51:43,473] Trial 33 finished with value: 0.9632914911872772 and parameters: {'n_estimators': 862, 'num_leaves': 420, 'max_depth': 37, 'learning_rate': 0.08714139722324872, 'min_split_gain': 0.009407188032293737, 'feature_fraction': 0.915144389050584, 'bagging_freq': 10}. Best is trial 30 with value: 0.9639839649258637.


[LightGBM] [Warning] feature_fraction is set=0.9446078365438996, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9446078365438996
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] feature_fraction is set=0.9446078365438996, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9446078365438996
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] feature_fraction is set=0.9446078365438996, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9446078365438996
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] feature_fraction is set=0.9446078365438996, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9446078365438996
[LightGBM] [Warning] bagging_freq is set=10, subsample

[I 2023-07-03 14:56:34,559] Trial 34 finished with value: 0.958051038927984 and parameters: {'n_estimators': 873, 'num_leaves': 377, 'max_depth': 39, 'learning_rate': 0.0716140415106059, 'min_split_gain': 0.009161946736916561, 'feature_fraction': 0.9446078365438996, 'bagging_freq': 10}. Best is trial 30 with value: 0.9639839649258637.


[LightGBM] [Warning] feature_fraction is set=0.9222570324996335, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9222570324996335
[LightGBM] [Warning] bagging_freq is set=9, subsample_freq=0 will be ignored. Current value: bagging_freq=9
[LightGBM] [Warning] feature_fraction is set=0.9222570324996335, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9222570324996335
[LightGBM] [Warning] bagging_freq is set=9, subsample_freq=0 will be ignored. Current value: bagging_freq=9
[LightGBM] [Warning] feature_fraction is set=0.9222570324996335, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9222570324996335
[LightGBM] [Warning] bagging_freq is set=9, subsample_freq=0 will be ignored. Current value: bagging_freq=9
[LightGBM] [Warning] feature_fraction is set=0.9222570324996335, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9222570324996335
[LightGBM] [Warning] bagging_freq is set=9, subsample_freq=0

[I 2023-07-03 15:01:13,274] Trial 35 finished with value: 0.9456563973261731 and parameters: {'n_estimators': 736, 'num_leaves': 425, 'max_depth': 27, 'learning_rate': 0.04979305564966731, 'min_split_gain': 0.013619776000698377, 'feature_fraction': 0.9222570324996335, 'bagging_freq': 9}. Best is trial 30 with value: 0.9639839649258637.


In [23]:
best_param = study.best_params
print(study.best_value)
best_param

0.9639839649258637


{'n_estimators': 732,
 'num_leaves': 476,
 'max_depth': 41,
 'learning_rate': 0.09696302667271771,
 'min_split_gain': 0.013955193706078131,
 'feature_fraction': 0.9080993113718123,
 'bagging_freq': 10}

In [27]:
model = LGBMClassifier(**best_param)

In [28]:
%%time
model.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.9080993113718123, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9080993113718123
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
CPU times: total: 4min 53s
Wall time: 43.5 s


LGBMClassifier(bagging_freq=10, feature_fraction=0.9080993113718123,
               learning_rate=0.09696302667271771, max_depth=41,
               min_split_gain=0.013955193706078131, n_estimators=732,
               num_leaves=476)

In [29]:
y_pred_val = model.predict(X_test)
accuracy_val = accuracy_score(y_test, y_pred_val)
print(f"Validation Accuracy: {accuracy_val}")

Validation Accuracy: 0.904445772673583


In [30]:
# model.booster_.save_model('../models/1-cls-07032023.txt')

In [32]:
loaded_model = lgb.Booster(model_file='../models/1-cls-07032023.txt')

In [42]:
y_pred_val = loaded_model.predict(X_test)
accuracy_val = accuracy_score(y_test, y_pred_val > 0.5)
print(f"Validation Accuracy: {accuracy_val}")

Validation Accuracy: 0.904445772673583
